In [ ]:
import sys, os, mosek

# Since the actual value of Infinity is ignored, we define it solely
# for symbolic purposes:
inf = 0.0

# Define a stream printer to grab output from MOSEK
def streamprinter(text):
    sys.stdout.write(text)
    sys.stdout.flush()


# Create a task
with mosek.Task() as task:
    task.set_Stream(mosek.streamtype.log, streamprinter)
    # Set up and input bounds and linear coefficients
    bkc = [mosek.boundkey.lo]
    blc = [1.0]
    buc = [inf]
    numvar = 3
    bkx = [mosek.boundkey.lo] * numvar
    blx = [0.0] * numvar
    bux = [inf] * numvar
    c = [0.0, -1.0, 0.0]
    asub = [[0], [0], [0]]
    aval = [[1.0], [1.0], [1.0]]

    numvar = len(bkx)
    numcon = len(bkc)

    # Append 'numcon' empty constraints.
    # The constraints will initially have no bounds.
    task.appendcons(numcon)

    # Append 'numvar' variables.
    # The variables will initially be fixed at zero (x=0).
    task.appendvars(numvar)

    for j in range(numvar):
        # Set the linear term c_j in the objective.
        task.putcj(j, c[j])
        # Set the bounds on variable j
        # blx[j] <= x_j <= bux[j]
        task.putvarbound(j, bkx[j], blx[j], bux[j])
        # Input column j of A
        task.putacol(j,                  # Variable (column) index.
                        # Row index of non-zeros in column j.
                        asub[j],
                        aval[j])            # Non-zero Values of column j.
    for i in range(numcon):
        task.putconbound(i, bkc[i], blc[i], buc[i])

    # Set up and input quadratic objective
    qsubi = [0, 1, 2, 2]
    qsubj = [0, 1, 0, 2]
    qval = [2.0, 0.2, -1.0, 2.0]

    task.putqobj(qsubi, qsubj, qval)

    # Input the objective sense (minimize/maximize)
    task.putobjsense(mosek.objsense.minimize)

    # Optimize
    task.optimize()
    # Print a summary containing information
    # about the solution for debugging purposes
    task.solutionsummary(mosek.streamtype.msg)

    prosta = task.getprosta(mosek.soltype.itr)
    solsta = task.getsolsta(mosek.soltype.itr)

    # Output a solution
    xx = task.getxx(mosek.soltype.itr)

    if solsta == mosek.solsta.optimal:
        print("Optimal solution: %s" % xx)
    elif solsta == mosek.solsta.dual_infeas_cer:
        print("Primal or dual infeasibility.\n")
    elif solsta == mosek.solsta.prim_infeas_cer:
        print("Primal or dual infeasibility.\n")
    elif mosek.solsta.unknown:
        print("Unknown solution status")
    else:
        print("Other solution status")

Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : QO (quadratic optimization problem)
  Constraints            : 1               
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 3               
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.00    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time    

In [ ]:
import os
os.system('clear')

import mosek
import mosek.fusion as mf

def solve_qp():
    # Create a MOSEK model
    M = mf.Model("Quadratic Program")

    # Define the number of variables
    n = 2

    # Define decision variables (continuous, unrestricted)
    x = M.variable("x", n, mf.Domain.unbounded())

    # Define the quadratic term matrix Q (must be positive semidefinite)
    Q = mf.Matrix.dense([[2.0, 0.5], [0.5, 1.0]])

    # Define the linear term vector c
    c = [-3.0, -1.0]

    # Define the constraint matrix A and constraint vector b
    A = [[1.0, 2.0], [2.0, 1.0]]
    b = [2.0, 2.0]

    # Define the quadratic term in the objective function
    q_expr = mf.Expr.mul(x,Q, x)
    #mf.Expr.dot(x, q_expr)
    #quad_expr = mf.Expr.dot(x, mf.Expr.mul(Q, x))

    # Define the linear term in the objective function
    lin_expr = mf.Expr.dot(c, x)

    # Set the objective: Minimize (1/2) * x^T Q x + c^T x
    #M.objective("objective", mf.ObjectiveSense.Minimize, mf.Expr.add(mf.Expr.mul(0.5, quad_expr), lin_expr))
    M.objective("objective", mf.ObjectiveSense.Minimize, lin_expr)


    # Define constraints Ax ≤ b
    M.constraint("constraints", mf.Expr.mul(A, x), mf.Domain.lessThan(b))

    # Solve the QP
    M.solve()

    # Get the optimal solution
    #solution = x.level()
    #print("Optimal solution:", solution)
    #print(x.Val)

    # Clean up
    #M.dispose()

# Run the solver
solve_qp()